## Working with Kevin
*date*: 8/16/2018

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Fuck warnings
import warnings
warnings.filterwarnings('ignore')

# mas
%matplotlib inline
from selenium import webdriver
from pandas import *
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import *

In [2]:
master_df = pd.DataFrame()

## Basketball Reference data

In [3]:
# Fetch Per 100 possession data
br_per100_url = "https://www.basketball-reference.com/leagues/NBA_2018_per_poss.html"
br_per100_data = pd.read_html(br_per100_url)[0]
br_per100_data.columns = [stat + "_BR_100" for stat in br_per100_data.columns.tolist()]

cols_100 = [elm for elm in br_per100_data.columns.tolist()[8:]]
cols_100.append( br_per100_data.columns.tolist()[1] )

In [4]:
# Append to master dataframe
for stat in cols_100:
    master_df[stat] = br_per100_data[stat] 

In [5]:
# Fetch Advanced data
br_adv_url  = "https://www.basketball-reference.com/leagues/NBA_2018_advanced.html"
br_adv_data = pd.read_html(br_adv_url)[0]
br_adv_data.columns = [stat + "_BR_ADV" for stat in br_adv_data.columns.tolist()]
cols_adv = br_adv_data.columns.tolist()[1:3] + br_adv_data.columns.tolist()[4:]

In [6]:
# Append to master dataframe
for stat in cols_adv:
    master_df[stat] = br_adv_data[stat]

In [7]:
# Fetch Total data
br_tot_url = "https://www.basketball-reference.com/leagues/NBA_2018_totals.html"
br_tot_data = pd.read_html(br_tot_url)[0]
br_tot_data.columns = [stat + "_BR_TOT" for stat in br_tot_data.columns.tolist()]

cols_list = br_tot_data.columns.tolist()
cols_tot = [stat for stat in cols_list[8:] if ( ('%' not in stat) or (stat == "eFG%_BR_TOT") )]
cols_tot.append( cols_list[1] )

In [8]:
# Append to master dataframe
for stat in cols_tot:
    master_df[stat] = br_tot_data[stat]

## NBA Stats

In [10]:
# Define URLs to scrape
url_0 = "http://stats.nba.com/players/usage/"
url_1 = "http://stats.nba.com/players/rebounding/"
url_2 = "http://stats.nba.com/players/opponent/?sort=GP&dir=-1"
url_3 = "http://stats.nba.com/players/defense/"
url_4 = "http://stats.nba.com/players/advanced/?sort=GP&dir=-1"
url_5 = "http://stats.nba.com/players/touches/"
url_6 = "http://stats.nba.com/players/speed-distance/"
url_7 = "http://stats.nba.com/players/defense-dash-overall/"

url_list = [url_0, url_1, url_2, url_3, url_4, url_5, url_6, url_7]

In [11]:
def fn( browser ):
    
    # Get table from HTML
    table = browser.find_element_by_class_name('nba-stat-table__overflow')
    
    player_ids = []
    player_names = []
    player_stats = []

    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id == 0:
            column_names = lines.split(' ')[1:]
        else:
            if line_id % 3 == 1:
                player_ids.append(lines)
            if line_id % 3 == 2:
                player_names.append(lines)
            if line_id % 3 == 0:
                player_stats.append( [float(i) for i in lines.split(' ')] )
              
    print("Made it here\n")
    db = pandas.DataFrame({'player': player_names,
                           'gp': [i[0] for i in player_stats],
                           'min': [i[1] for i in player_stats],
                           'pts': [i[2] for i in player_stats],
                           'fgm': [i[3] for i in player_stats], 
                           'fga': [i[4] for i in player_stats],
                           'fg%': [i[5] for i in player_stats],
                           '3pm': [i[6] for i in player_stats],
                           '3pa': [i[7] for i in player_stats],
                           '3p%': [i[8] for i in player_stats],
                           'ftm': [i[9] for i in player_stats],
                           'fta': [i[10] for i in player_stats],
                           'ft%': [i[11] for i in player_stats],
                           'oreb': [i[12] for i in player_stats],
                           'dreb': [i[13] for i in player_stats],
                           'reb': [i[14] for i in player_stats],
                           'ast': [i[15] for i in player_stats],
                           'stl': [i[16] for i in player_stats],
                           'blk': [i[17] for i in player_stats],
                           'tov': [i[18] for i in player_stats],
                           'eff': [i[19] for i in player_stats]
                           }
                         )
    
    db = db[['player', 
         'gp', 
         'min', 
         'pts', 
         'fgm', 
         'fga', 
         'fg%', 
         '3pm', 
         '3pa', 
         '3p%', 
         'ftm',
         'fta', 
         'ft%', 
         'oreb', 
         'dreb',
         'reb',
         'ast',
         'stl',
         'blk',
         'tov',
         'eff']
      ]

    return db

In [12]:
import sys
db_list = list()
for url in url_list:
    
    print("URL: ", url)
    
    ## Set up chrome driver 
    path_to_chromedriver = "./web_scraping/chromedriver"
    browser = webdriver.Chrome(executable_path=path_to_chromedriver)
    browser.get(url)
    
    try:
    
        ## This is for the correct seasson year
        browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[2]').click()
        ## This is for "all" players option 
        browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/select/option[1]').click()

        ## Follow up, just need to prompt user to enter the season you want to look at and such.

        db = fn( browser )
        db_list( db )
    
    except:
        browser.close()
        print("Unexpected error:", sys.exc_info()[0])

URL:  http://stats.nba.com/players/usage/
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
URL:  http://stats.nba.com/players/rebounding/
Unexpected error: <class 'ValueError'>
URL:  http://stats.nba.com/players/opponent/?sort=GP&dir=-1
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
URL:  http://stats.nba.com/players/defense/
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
URL:  http://stats.nba.com/players/advanced/?sort=GP&dir=-1
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
URL:  http://stats.nba.com/players/touches/
Unexpected error: <class 'ValueError'>
URL:  http://stats.nba.com/players/speed-distance/
Unexpected error: <class 'ValueError'>
URL:  http://stats.nba.com/players/defense-dash-overall/
Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
